Quadrupedal Walking Frog Code

This code analyses the force and kinematic data for multiple frog species walked across a multi-axis force platform. The x, y, and z forces were analysed. Using the x, y and z force data, we calculate the resultant force (). Joint angle data was collect via digitization of high-speed videography in two views (dorsal and lateral) over two strides for the fore- and hindlimbs.
Contributors:
J Fabian Dubon
M Kearns
CM Reynaga
Last updated:
Dec 28, 2022 /April 5, 2024




Setting Directory 

In [423]:
import pandas as pd 
import numpy as np 
import openpyxl as pyxl  
 
#'/Users/jenifer_dubon/Desktop/walking_frog_force/walking_frog_force/Datasets/walks_workbook.xlsx'


Parameters
A single stride was defined as: The start of the stride begins the moment the forelimb phallange/limb closest to the lateral veiew camera makes contact with the ground. The stride ends once all limbs have cycled through, until the start of the same forelimb is about to make contact again with the ground to begin a new stride.
Video Information
This section will read in the parameters from the following imported excel sheet.
Defines a variable with the file name as a string

In [424]:
params = '/Users/jenifer_dubon/Desktop/walking_frog_force/walking_frog_force/Datasets/walks_workbook.xlsx'


Imports data file and saves the file as a struct named parameters, the struct is organized as within two fields (1) data and (2) textdata. Any column names or text values within the spreadsheet will be in textdata and any numerical values will be in data.

In [425]:
parameters = pd.read_excel(params)

elects which row to process within the excel parameters sheet
Change number based upon the row you'd like to process within the excel spread sheet and save the matlab code before you run the code

In [426]:
excel_row_to_process = 2 #must start at 2 bc of pandas structure
film_side_right = True

Video/Force Sampling Information
Reading in parameters from excel sheet.


In [427]:
row_to_process = excel_row_to_process-1

# Print the size of the parameters variable
display(parameters.shape)

if 'Date' in parameters.columns:
    date = str(int(parameters['Date'].iloc[row_to_process])) #iloc is more of a personal pref #must change to int first to get rid off .0 
else:
    print("Error: 'Date' not found in DataFrame")

genus_species = 'pb' # Phrynomantis bifasciatus

if 'Ind.' in parameters.columns:
    individual = str(int(parameters['Ind.'].iloc[row_to_process]))
else:
    print("Error: 'Ind.' not found in DataFrame")

if 'Walk' in parameters.columns:
    walk_num = str(int(parameters['Walk'].iloc[row_to_process]))
else:
    print("Error: 'Walk' not found in DataFrame")

print(walk_num)
walk_seq = date + '_' + genus_species + individual + '_walk'+ walk_num 
fps = 250

print(walk_seq)


(62, 30)

2
20210706_pb4_walk2


In [428]:

""" this is for me to try different approaches.


row_to_process = excel_row_to_process-1

# Print the size of the parameters variable
display(parameters.shape)

type(parameters) #gets the type 

if 'Date' in parameters.columns:
    date = str(parameters['Date'].iloc[row_to_process])

else:
    print("Error: Date not found in DataFrame")
    
print(date)
    
if 'Ind.' in parameters.columns:
    individual = str(parameters['Ind.'][row_to_process])

else:
    print('Error')    
    
print(individual)

type(individual)


#set up variables to right columns 
#date = str(parameters.Date[row_to_process])


walk_seq = date,'_', genus_species, individual, '_walk', walk_num #append has a limited num of arguments
#so the best way to concatenate strings is to add the '+ ' sign in between 
#print(date)

#type(date)
#parameters.info() #gets the """



' this is for me to try different approaches.\n\n\nrow_to_process = excel_row_to_process-1\n\n# Print the size of the parameters variable\ndisplay(parameters.shape)\n\ntype(parameters) #gets the type \n\nif \'Date\' in parameters.columns:\n    date = str(parameters[\'Date\'].iloc[row_to_process])\n\nelse:\n    print("Error: Date not found in DataFrame")\n    \nprint(date)\n    \nif \'Ind.\' in parameters.columns:\n    individual = str(parameters[\'Ind.\'][row_to_process])\n\nelse:\n    print(\'Error\')    \n    \nprint(individual)\n\ntype(individual)\n\n\n#set up variables to right columns \n#date = str(parameters.Date[row_to_process])\n\n\nwalk_seq = date,\'_\', genus_species, individual, \'_walk\', walk_num #append has a limited num of arguments\n#so the best way to concatenate strings is to add the \'+ \' sign in between \n#print(date)\n\n#type(date)\n#parameters.info() #gets the '

In [429]:
#frames per second (fps) high speed video frame rate
igor_sr = 10000

if 'Ind. Weight (g)' in parameters.columns:
    body_mass = parameters['Ind. Weight (g)'].iloc[row_to_process]
else:
    print("Error: 'Ind. Weight (g)' not found in DataFrame")

print(body_mass) #prints are to make sure results are the same

5.77


 *Video Conditions*
 Determines whether the code should run data on the forelimb or the hindlimb.
 A value of 1 will run analysis for the relevent limbs, however a value of 0
 will skip those sections for analysis within the script. Additionally, |Overlap|
 is used to distinguish when the fore- and hindlimb are both in contact with
 the force plate, and thus individual limb analysis during that time can not
 be done.

In [430]:
#these variables  are floats so they got 0.0 
if 'Forelimb Status (0/1)' in parameters.columns:
    FL = parameters['Forelimb Status (0/1)'].iloc[row_to_process]
else:
    print("Error: 'Forelimb Status (0/1)' not found in DataFrame")

print(FL)

if 'Hindlimb Status (0/1)' in parameters.columns:
    HL = parameters['Hindlimb Status (0/1)'].iloc[row_to_process]
else:
    print("Error: 'Hindlimb Status (0/1)' not found in DataFrame")

print(HL)

if 'Overlap (0/1)' in parameters.columns:
    Overlap = parameters['Overlap (0/1)'].iloc[row_to_process]
else:
    print("Error: 'Overlap (0/1)' not found in DataFrame")
print(Overlap)



1.0
0.0
1.0


Walk Sequence Information
Converting video frames into time (sec)

In [431]:
if 'tot. dltdv8 frames' in parameters.columns:
    totalf = parameters['tot. dltdv8 frames'].iloc[row_to_process] # total amount of video frames
else:
    print("Error: 'tot. dltdv8 frames' not found in DataFrame")
print(totalf)

678.0


In [432]:
#totalf_sec is rounded here to the same decimals place as the force data frequency
totalf_sec = round((totalf/fps),4) #total amount of time filmed (sec)
print(totalf_sec)

2.712


Defining Stride 1
Beginning frame for stride 1 defined when forelimb (or phalange of the forelimb) nearest to the camera touches the ground


In [433]:
if 'Forelimb Start frame' in parameters.columns:
    startf_s1 = parameters['Forelimb Start frame'].iloc[row_to_process]-1 #bc of indexing in python, it starts at 97 not 98 (ensures the pt1_X are the same in matlab and python)
else:
    print("Error: 'Forelimb Start frame' not found in DataFrame")
print(startf_s1)
print(row_to_process)

startf_s1_sec = round((startf_s1/fps),4)
print(startf_s1_sec)


97.0
1
0.388


End frame for stride 1 
Defined as the frame right before the forelimb touches back to start the next stride

In [434]:
if 'Stride 1 end frame' in parameters.columns:
    endf_s1 = parameters['Stride 1 end frame'].iloc[row_to_process]-1 #same here, it ends at 294
else:
    print("Error: 'Stride 1 end frame' not found in DataFrame")
print(endf_s1)

294.0


Conversion from frames to time (sec) by dividing by frame rate or recording frequency in frames per second (fps).

time(sec) = frame #/fps

In [435]:
endf_s1_sec = round((endf_s1/fps),4)
print(endf_s1_sec)

1.176


Defining Stride 2
 See section above for more information on defining a stride and calculating
 time from frame number for a stride.


In [436]:
if 'Stride 2 Start frame' in parameters.columns:
    startf_s2 = parameters['Stride 2 Start frame'].iloc[row_to_process]
else:
    print("Error: 'Stride 2 Start frame' not found in DataFrame")
print(startf_s2)

startf_s2_sec = round((startf_s2/fps),4)
print(startf_s2_sec)

296.0
1.184


End frame for stride 2;
If not a complete 2ns stride insert '0' within parameters excel sheet (however at this point we should only be analyzing walks with 2 complete strides).

In [437]:
if 'Stride 2 end frame' in parameters.columns:
    endf_s2 = parameters['Stride 2 end frame'].iloc[row_to_process]
else:
    print("Error: 'Stride 2 end frame' not found in DataFrame")
print(endf_s2)

endf_s2_sec = round((endf_s2/fps),4)
print(endf_s2_sec)

0
0.0


Characterizing Igor time based upon trigger time
 Finding when the cameras and data collection in Igor were triggered off.
 format long g % if not displaying full decimal


In [438]:
if 'Igor trigger time' in parameters.columns:
    trigger_row = parameters['Igor trigger time'].iloc[row_to_process]
else:
    print("Error: 'Igor trigger time' not found in DataFrame")
print(trigger_row)

trigger_t = trigger_row/10000
print(trigger_t)

8714.0
0.8714


Creating a time array for forceplate data. 
 Igor|_sr| = sampling rate. Plus rounding to the correct decimals place based upon the Igor sampling frequency (1000) force recording for a total of 10 seconds.

In [439]:
force_t = np.arange(1, igor_sr + 1)/1000

force_time = np.round(force_t, 4)
print(force_time.size)

10000


FL and HL Contact Timing
  Defining relative to time when the forelimb and hindlimb are in contact with  the force plate.

 |fl_end| = last frame the front finger/hand is in contact with the force plate before leaving

In [440]:
if 'Forelimb end frame' in parameters.columns:
    fl_end = parameters['Forelimb end frame'].iloc[row_to_process]
else:
    print("Error: 'Forelimb end frame' not found in DataFrame")
print(fl_end)

fl_end_sec = round((fl_end/fps),4); # converting to time
print(fl_end_sec)

270.0
1.08


hl_start:
 when a toe/foot touches the force plate/hindlind makes contact with the forceplate for the first time.

In [441]:
if 'Hindlimb Start frame' in parameters.columns:
    hl_start = parameters['Hindlimb Start frame'].iloc[row_to_process]
else:
    print("Error: 'Hindlimb Start frame' not found in DataFrame")
print(hl_start)

hl_start_sec = round((hl_start/fps),4); # converting to time
print(hl_start_sec)

239.0
0.956


hl_end:
 last frame back toe touches force plate before leaving

In [442]:
if 'Hindlimb end frame' in parameters.columns:
    hl_end = parameters['Hindlimb end frame'].iloc[row_to_process]
else:
    print("Error: 'Hindlimb end frame' not found in DataFrame")
print(hl_end)

hl_end_sec = round((hl_end/fps),4); # converting to time
print(hl_end_sec)

0.0
0.0


Determining Periods of Overlap
 Isolating FL and HL contact only periods
 During most walks the FL and HL will both be in contact with the force plate
 midway between stride 1 and stride 2. Therefore, we only analyze the forces
 recorded when and individual limb is in contact with the force platform. More
 specifically, we only analyze the forelimb contact period during the time before
 the hindlimb touches down. And analyze the hindlimb contact during the time
 after the forelimb has left the force plate.

 |fl_only_end| = frame where only the FL leaves contact with the force plate,
 then converting to time (sec).


In [443]:
# MAYBE DO TODO will need a conditional statement for walks where there isn't overlap
fl_only_end = hl_start-1
print(fl_only_end)
fl_only_end_sec = round((fl_only_end/fps),4)
print(fl_only_end_sec)


238.0
0.952


hl_only_start:
 start frame where the HL is the only one that is in contact with the force plate, then converting frames to time (sec).

In [444]:
hl_only_start = fl_end+1
print(hl_only_start)
hl_only_start_sec = round((hl_only_start/fps),4)
print(hl_only_start_sec)

271.0
1.084


Changing reference frame of the video land markers

Finding the distance in time from the end of the video, to set-up and align time relative to the trigger time. And ultimately determine where force occurs relative to trigger and video timing. Example conversion equations provided:

    time = t_video_length - t_event_start

In [445]:
delta_s1_start = totalf_sec - startf_s1_sec
print(delta_s1_start)
delta_hl_on = totalf_sec-hl_start_sec
print(delta_hl_on)
delta_fl_end = totalf_sec-fl_end_sec
print(delta_fl_end)
delta_s1_end = totalf_sec-endf_s1_sec
print(delta_s1_end)
delta_s2_start = totalf_sec-startf_s2_sec
print(delta_s2_start)
delta_hl_end = totalf_sec-hl_end_sec
print(delta_hl_end)
delta_s2_end = totalf_sec-endf_s2_sec
print(delta_s2_end)

delta_fl_only_end = totalf_sec - fl_only_end_sec
print(delta_fl_only_end)
delta_hl_only_start = totalf_sec - hl_only_start_sec
print(delta_hl_only_start)


2.3240000000000003
1.7560000000000002
1.6320000000000001
1.5360000000000003
1.5280000000000002
2.712
2.712
1.7600000000000002
1.6280000000000001


Importing Digitized Data Points

All videos for K. senegalensis were digitized with the assigned point number values.
FIXIT Note: this will differ across different species unfortunately.
Eye
Elbow
Wrist
Shoulder
Knee
Ankle
Hip
Knee
Tarsometatarsal (HL)
Tarsometatarsal (LF)

Importing digitized XYZ pts spreadsheet
Imports xyz points as a struct/tuple named xyzpts

In [446]:
xyzfile = walk_seq + '_xyzpts.csv'
xyzfile = '/Users/jenifer_dubon/Desktop/walking_frog_force/walking_frog_force/Datasets/' + xyzfile
print('/Users/jenifer_dubon/Desktop/walking_frog_force/walking_frog_force/Datasets/' + xyzfile)
xyzpts = pd.read_csv(xyzfile)

print(xyzpts.shape) #different than matalab (matlab = 311x30, but this is the same as excel sheet)

print(type(xyzpts.shape))

/Users/jenifer_dubon/Desktop/walking_frog_force/walking_frog_force/Datasets//Users/jenifer_dubon/Desktop/walking_frog_force/walking_frog_force/Datasets/20210706_pb4_walk2_xyzpts.csv
(678, 30)
<class 'tuple'>


Assigning points as variables from the imported spreadsheet matrix xyzpts struct/Tuple


In order to make sure that MATLAB orients the data correctly, we assign the point values to the individual columns, and specify the frame number range separated by a column that we want to grab data from. Importanting relative to stride 1 and 2. The number designates the excel sheet row to look for.x

In [480]:
#display(xyzpts)
print( startf_s1)
print(endf_s1)
print(type( xyzpts.iloc[1].shape))

for col in xyzpts.columns:
    row_section_s1 = xyzpts[col][int(startf_s1):int(endf_s1) + 1]
    #display(row_section_s1)
    if 'pt1_X' in col:
        pt1_X_s1 = row_section_s1 
    elif 'pt1_Y' in col:
        pt1_Y_s1 = row_section_s1
    elif 'pt1_Z' in col:
        pt1_Z_s1 = row_section_s1
        
    elif 'pt2_X' in col:
        pt2_X_s1 = row_section_s1 
    elif 'pt2_Y' in col:
        pt2_Y_s1 = row_section_s1
    elif 'pt2_Z' in col:
        pt2_Z_s1 = row_section_s1 
        
    elif 'pt3_X' in col:
        pt3_X_s1 = row_section_s1
    elif 'pt3_Y' in col:
        pt3_Y_s1 = row_section_s1
    elif 'pt3_Z' in col:
        pt3_Z_s1 = row_section_s1  
        
    elif 'pt4_X' in col:
        pt4_X_s1 = row_section_s1     
    elif 'pt4_Y' in col:
        pt4_Y_s1 = row_section_s1
    elif 'pt4_Z' in col:
        pt4_Z_s1 = row_section_s1
    
    elif 'pt5_X' in col:
        pt5_X_s1 = row_section_s1
    elif 'pt5_Y' in col:
        pt5_Y_s1 = row_section_s1
    elif 'pt5_Z' in col:
        pt5_Z_s1 = row_section_s1
    
    elif 'pt6_X' in col:
        pt6_X_s1 = row_section_s1
    elif 'pt6_Y' in col:
        pt6_Y_s1 = row_section_s1
    elif 'pt6_Z' in col:
        pt6_Z_s1 = row_section_s1
    
    elif 'pt7_X' in col:
        pt7_X_s1 = row_section_s1
    elif 'pt7_Y' in col:
        pt7_Y_s1 = row_section_s1
    elif 'pt7_Z' in col:
        pt7_Z_s1 = row_section_s1
        
    elif 'pt8_X' in col:
        pt8_X_s1 = row_section_s1
    elif 'pt8_Y' in col:
        pt8_Y_s1 = row_section_s1
    elif 'pt8_Z' in col:
        pt8_Z_s1 = row_section_s1
        
    elif 'pt9_X' in col:
        pt9_X_s1 = row_section_s1
    elif 'pt9_Y' in col:
        pt9_Y_s1 = row_section_s1
    elif 'pt9_Z' in col:
        pt9_Z_s1 = row_section_s1
        
    elif 'pt10_X' in col:
        pt10_X_s1 = row_section_s1
    elif 'pt10_Y' in col:
        pt10_Y_s1 = row_section_s1
    elif 'pt10_Z' in col:
        pt10_Z_s1 = row_section_s1
        
print(xyzpts.columns)
display(row_section_s1)
#print(row_section) check the sections are correct
display(pt2_Z_s1)


97.0
294.0
<class 'tuple'>
Index(['pt1_X', 'pt1_Y', 'pt1_Z', 'pt2_X', 'pt2_Y', 'pt2_Z', 'pt3_X', 'pt3_Y',
       'pt3_Z', 'pt4_X', 'pt4_Y', 'pt4_Z', 'pt5_X', 'pt5_Y', 'pt5_Z', 'pt6_X',
       'pt6_Y', 'pt6_Z', 'pt7_X', 'pt7_Y', 'pt7_Z', 'pt8_X', 'pt8_Y', 'pt8_Z',
       'pt9_X', 'pt9_Y', 'pt9_Z', 'pt10_X', 'pt10_Y', 'pt10_Z'],
      dtype='object')


97     76.432015
98     76.829474
99     78.047407
100    78.663691
101    79.346709
         ...    
290    77.263047
291    77.951807
292    78.667312
293    79.383035
294    79.913944
Name: pt10_Z, Length: 198, dtype: float64

97     73.954070
98     73.936965
99     73.965687
100    74.061839
101    74.689817
         ...    
290    75.110863
291    75.562327
292    75.944462
293    76.058882
294    76.427859
Name: pt2_Z, Length: 198, dtype: float64

If data set has hindlimb data


In [495]:
print('startf_s2 =', startf_s2)
print('endf_s2 =', endf_s2)
print

#display(xyzpts.columns)


if HL == 1:
    for col in xyzpts.columns:
        row_section_s2 = xyzpts[col][int(startf_s2):int(endf_s2) + 1]
        if 'pt1_X' in col:
            pt1_X_s2 = row_section_s2 
            
    print(row_section_s2)
        #if 'pt1_'

startf_s2 = 296.0
endf_s2 = 0
